## RANDOM FORESTS using only NumPy and Pandas

### by Shailesh Rao

In [1]:
import numpy as np
import pandas as pd

In [28]:
import random

# class for the node of the decision tree
class Node:
    def __init__(self, feature=None, impurity=None, thresh=None, left_node=None, right_node=None, value=None, isLeaf=False):
        #feature based on which the node splits the data
        self.feature = feature

        # gini impurity of the node
        self.impurity = impurity

        # threshold of self.value to decide splits
        self.thresh = thresh

        self.left_node = left_node
        self.right_node = right_node

        # value or final label predicted if the node is a leaf node
        self.value = value

        # whether node is a leaf
        self.isLeaf = isLeaf

# class for decision tree
class DecisionTree:
    def __init__(self, max_depth=3, min_rows_split=3):
        # minimum number of rows required for the data to be split
        self.min_rows_split = min_rows_split

        # maximum depth of the tree
        self.max_depth = max_depth

        # root node of the tree
        self.root = None
    
    def fit(self, X, y):
        '''
        fits the tree with given training data
        '''
        df = pd.concat([X, y], axis = 1)
        self.root = self.make_tree(0, df)
    
    def make_tree(self, depth, df):
        '''
        recursively builds the decision tree
        '''
        rows = len(df.iloc[:,:-1])
        features = list(df.iloc[:,:-1].columns)

        # condition to split the current node
        if self.max_depth > depth and rows > self.min_rows_split:
            best_split = self.optimal_feature(df)

            # if the node is still impure, it is split
            if best_split['gini'] > 0:
                l_tree = self.make_tree(depth+1, best_split['left_split'])
                r_tree = self.make_tree(depth+1, best_split['right_split'])
                return Node(best_split['feat'], best_split['gini'], best_split['thresh'], l_tree, r_tree)
        print('df:\n', df,'\n')

        # final label the node predicts, i.e if it is a leaf
        final_value = self.final_label(df.iloc[:,-1])
        return Node(value=final_value, isLeaf=True)

    def optimal_feature(self, df):
        '''
        finds the optimal split (with minimum gini impurity)
        '''
        X = df.iloc[:,:-1]
        y = df.iloc[:,-1]
        features = list(X.columns)
        min_gini = float("inf")

        # iterating through features
        for feature in features:
            values = df[feature].unique()

            # iterating through all possible thresholds
            for val in values:
                l_condition = df[feature] <= val
                r_condition = df[feature] > val
                l_df = df[l_condition]
                r_df = df[r_condition]

                # finds weighted impurity of current node based on possible child nodes
                gini = self.weighted_gini_impurity(l_df, r_df)

                # if it is lower than current gini impurity, update the best split values
                if gini < min_gini:
                    min_gini = gini
                    result = {
                        'feat': feature,
                        'thresh': val,
                        'left_split': l_df,
                        'right_split': r_df,
                        'gini': gini
                    }
        return result

    def weighted_gini_impurity(self, l, r):
        ''' 
        finds weighted impurity of current node based on possible child nodes
        '''
        weight_r = len(r)/(len(l)+len(r))
        weight_l = 1 - weight_r
        return (weight_r*(self.gini_impurity(r)) + weight_l*(self.gini_impurity(l)))

    def gini_impurity(self, subset):
        ''' 
        calculates gini impurity based on formula (1 - summation((probability of labels) ^ 2))
        '''
        target  = subset.iloc[:,-1]
        sub_label_distr = dict(target.value_counts())
        total = len(target)
        sum_squares = 0
        for label in sub_label_distr.keys():
            sum_squares += (sub_label_distr[label]/total)**2
        impurity = 1 - sum_squares
        return impurity
    
    def final_label(self, col):
        ''' 
        finds final label which leaf node predicts
        '''
        return (dict(map(reversed, dict(col.value_counts()).items()))[max(dict(col.value_counts()).values())])

    def predict_sub(self, input, tree):
        ''' 
        predicts label for a single test sample
        '''
        #if leaf node, return value contained
        if tree.isLeaf:
            return tree.value

        # if not a leaf node, traverses down the tree based on thresholds and features (recursively)
        else:
            limit = tree.thresh
            feat = tree.feature
            if input[feat]<=limit:
                return self.predict_sub(input, tree.left_node)
            else:
                return self.predict_sub(input, tree.right_node)
    
    def predict(self, inputs):
        ''' 
        predicts values for a set of test data
        '''
        preds = []
        for i in range(len(inputs)):
            preds.append(self.predict_sub(inputs.iloc[i], self.root))
        return preds

# class for random forest classifier
class RandomForest:
    def __init__(self, n_estimators):

        # number of decision trees in the forest
        self.n_estimators = n_estimators

        # array of decision trees
        self.forest = None

        # set of possible labels
        self.prediction_labels = None

    def create_forest(self, df):
        '''
        creates multiple trees for each bootstrapped dataset
        '''
        datasets = self.get_all_bootstraps(df)
        trees = []
        for dataset in datasets:
            tree = DecisionTree()
            tree.fit(dataset.iloc[:,:-1], dataset.iloc[:, -1])
            trees.append(tree)
        return trees

    def fit(self, X, y):
        ''' 
        fits the forest with given training data
        '''
        df = pd.concat([X, y], axis = 1)
        self.prediction_labels = list(y.unique())
        self.forest = self.create_forest(df)

    def bootstrap_dataset(self, df):
        ''' 
        creates a bootstrapped dataset from given training data
        '''
        boot_indices = []
        for i in range(len(df)):
            boot_indices.append(random.randint(0, len(df) - 1))
        boot_df = pd.DataFrame()
        for index in boot_indices:
            boot_df = pd.concat([boot_df, df.iloc[index]], axis = 1)  
        boot_df = boot_df.transpose()
        return boot_df

    def predict_matrix(self, inputs):
        ''' 
        creates a matrix of all the predictions by each tree in the forest
        '''
        predictions_df = pd.DataFrame()
        for tree in self.forest:
            preds_by_tree = tree.predict(inputs)
            predictions_df = pd.concat([predictions_df, pd.Series(preds_by_tree)], axis = 1)
        return predictions_df.transpose()

    def predict(self, inputs):
        ''' 
        predicts for a set of data
        '''
        matrix = self.predict_matrix(inputs)
        final_preds = []
        for tree in matrix.columns:
            final_preds.append(self.max_dict(dict((matrix[tree].value_counts()))))
        return final_preds

    def max_dict(self, d):
        ''' 
        gives key containing maximum value of a dictionary
        '''
        rev = dict(map(reversed, d.items()))
        return rev[max(list(d.values()))]

    def get_all_bootstraps(self, df):
        ''' 
        gets self.n_estimators number of bootstrapped datasets for each tree to train on
        '''
        bootstraps = []
        features = list(df.iloc[:,:-1].columns)
        for i in range(self.n_estimators):
            bootstraps.append({
                'df': self.bootstrap_dataset(df),
                'features': list(np.random.choice(features, size = int(np.ceil(np.sqrt(len(features)))), replace=False))+[df.columns[-1]]
            })
        bootstrapped_datasets = []
        for bootstrap in bootstraps:
            data_frame = bootstrap['df']
            feats = bootstrap['features']
            bootstrapped_datasets.append(data_frame[feats])
        return bootstrapped_datasets

In [3]:
#obtaining the genre dataset
df = pd.read_csv(r"datasets\genre_df.csv", sep = '\t');
df.drop(["Unnamed: 0"], inplace = True, axis = 1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1339 entries, 0 to 1338
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      1339 non-null   float64
 1   energy            1339 non-null   float64
 2   key               1339 non-null   int64  
 3   loudness          1339 non-null   float64
 4   mode              1339 non-null   int64  
 5   speechiness       1339 non-null   float64
 6   acousticness      1339 non-null   float64
 7   instrumentalness  1339 non-null   float64
 8   liveness          1339 non-null   float64
 9   valence           1339 non-null   float64
 10  tempo             1339 non-null   float64
 11  duration_ms       1339 non-null   int64  
 12  time_signature    1339 non-null   int64  
 13  track_name        1339 non-null   object 
 14  artist            1339 non-null   object 
 15  genre             1339 non-null   object 
dtypes: float64(9), int64(4), object(3)
memory 

In [5]:
df.drop(["artist", "track_name", "liveness", "key", "time_signature", "duration_ms"], axis = 1, inplace= True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1339 entries, 0 to 1338
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      1339 non-null   float64
 1   energy            1339 non-null   float64
 2   loudness          1339 non-null   float64
 3   mode              1339 non-null   int64  
 4   speechiness       1339 non-null   float64
 5   acousticness      1339 non-null   float64
 6   instrumentalness  1339 non-null   float64
 7   valence           1339 non-null   float64
 8   tempo             1339 non-null   float64
 9   genre             1339 non-null   object 
dtypes: float64(8), int64(1), object(1)
memory usage: 104.7+ KB


In [7]:
from sklearn.metrics import classification_report

In [8]:
# only few trees, because dataset is large

forest = RandomForest(n_estimators=5)

In [9]:
from sklearn.model_selection import train_test_split

X = df.drop(['genre'], axis = 1)
y = df['genre']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [10]:
forest.fit(X_train, y_train)

# printing the data contained in the final leaves
# the 'genre' column can be observed to have majorly one class (but some labels are misclassified)

df:
      valence    tempo loudness      genre
756    0.262   48.274  -31.414  CLASSICAL
981    0.327   82.456  -26.698       LOFI
1006    0.04  121.128  -20.688       LOFI
648    0.964  143.772   -22.62  CLASSICAL
754    0.543     84.4  -25.996  CLASSICAL
...      ...      ...      ...        ...
746    0.466  129.194  -22.589  CLASSICAL
772     0.82  148.018  -23.517  CLASSICAL
580    0.443   69.989  -35.424  CLASSICAL
643    0.122   80.633  -36.498  CLASSICAL
806    0.936  108.005  -26.624  CLASSICAL

[184 rows x 4 columns] 

df:
      valence    tempo loudness      genre
1031   0.593  159.574  -23.507       LOFI
649    0.286  167.216  -26.748  CLASSICAL
691    0.203  166.294   -25.45  CLASSICAL
765     0.18  182.502  -23.749  CLASSICAL
963    0.126  179.855  -27.295       LOFI
799    0.957  167.999  -27.084  CLASSICAL
963    0.126  179.855  -27.295       LOFI
691    0.203  166.294   -25.45  CLASSICAL
1030   0.436  160.033   -29.57       LOFI
1031   0.593  159.574  -23.507       LOF

In [11]:
preds = forest.predict(X_test)

In [12]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, preds))
print(confusion_matrix(y_test, preds))

              precision    recall  f1-score   support

   CLASSICAL       0.81      0.96      0.88        67
        LOFI       0.92      0.68      0.78        66
         POP       0.59      0.69      0.64        62
         RAP       0.75      0.76      0.75        74
        ROCK       0.85      0.76      0.80        66

    accuracy                           0.77       335
   macro avg       0.78      0.77      0.77       335
weighted avg       0.78      0.77      0.77       335

[[64  3  0  0  0]
 [14 45  2  4  1]
 [ 0  1 43 10  8]
 [ 1  0 17 56  0]
 [ 0  0 11  5 50]]


### Showing how model works for other smaller datasets

In [14]:
# iris dataset
iris = pd.read_csv('datasets/iris.csv')

In [15]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris.drop('species', axis = 1), iris['species'], test_size=0.33)

forest1 = RandomForest(n_estimators=100)

forest1.fit(X_train, y_train)

preds = forest1.predict(X_test)



In [23]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        20
  versicolor       1.00      0.76      0.87        17
   virginica       0.76      1.00      0.87        13

    accuracy                           0.92        50
   macro avg       0.92      0.92      0.91        50
weighted avg       0.94      0.92      0.92        50



In [24]:
#breast cancer dataset

data = pd.read_csv('datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mean_radius      569 non-null    float64
 1   mean_texture     569 non-null    float64
 2   mean_perimeter   569 non-null    float64
 3   mean_area        569 non-null    float64
 4   mean_smoothness  569 non-null    float64
 5   diagnosis        569 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 26.8 KB


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('diagnosis', axis = 1), data['diagnosis'], test_size=0.33)

forest1 = RandomForest(n_estimators=75)

forest1.fit(X_train, y_train)

preds = forest1.predict(X_test)



In [27]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.94      0.83      0.88        76
           1       0.89      0.96      0.93       112

    accuracy                           0.91       188
   macro avg       0.92      0.90      0.90       188
weighted avg       0.91      0.91      0.91       188

